In [1]:
import re
import sys
import csv
import pandas as pd
from collections import Counter

PATHS:

In [2]:
#hate speech:
IRONY_RAW_PATH = '../datasets/irony/raw/'
IRONY_INTERIM_PATH = '../datasets/irony/interim/'

#emoji:
EMOJI_RAW_PATH = '../datasets/emoji/raw/'
EMOJI_INTERIM_PATH = '../datasets/emoji/interim/'

FILES:

In [3]:
TEST = 'test_text.txt'
TRAIN = 'train_text.txt'
VAL = 'val_text.txt'

TEST_LABELS = 'test_labels.txt'
TRAIN_LABELS = 'train_labels.txt'
VAL_LABELS = 'val_labels.txt'


TEST_INTERIM = 'test_seperated.csv'
TRAIN_INTERIM = 'train_seperated.csv'
VAL_INTERIM = 'val_seperated.csv'

FUNCTIONS:

In [4]:
def tokenise(line):
    # Initialise lists
    tokens = []
    unmatchable = []

    # Compile patterns for speedup
    token_pat = re.compile(r'\w+|#|\'|@')
    skippable_pat = re.compile(r'[.,]+')  # typically spaces

    # As long as there's any material left...
    while line:
        # Try finding a skippable token delimiter first.
        skippable_match = re.search(skippable_pat, line)
        if skippable_match and skippable_match.start() == 0:
            # If there is one at the beginning of the line, just skip it.
            line = line[skippable_match.end():]
        else:
            # Else try finding a real token.
            token_match = re.search(token_pat, line)
            #print(token_match)
            if token_match and token_match.start() == 0:
                #print(line[token_match.start():token_match.end()])
                if line[token_match.start():token_match.end()] == '#': #keep hash tags together and seperate
                    try:
                        token_match2 = re.search(token_pat, line[1:])
                        tokens.append(line[:token_match2.end()+1])
                        line = line[token_match2.end()+1:]
                    except:
                        line = line[token_match.end():]

                elif line[token_match.start():token_match.end()] == '@': #keep @ tags together and seperate
                    try:
                        token_match2 = re.search(token_pat, line[1:])
                        if ' ' in line[token_match2.start():token_match2.end()]:
                            line = line[token_match.end():]
                        
                        else: 
                            tokens.append(line[:token_match2.end()+1])
                            line = line[token_match2.end()+1:]
                    except:
                        line = line[token_match.end():]

                elif line[token_match.start():token_match.end()] == "'": #handle contractions as a single word
                    try:
                        token_match2 = re.search(token_pat, line[1:])
                        if ' ' in line[token_match2.start():token_match2.end()]:
                            line = line[token_match.end():]
                        
                        else: 
                            tokens.append(line[:token_match2.end()+1])
                            line = line[token_match2.end()+1:]
                    except:
                        line = line[token_match.end():]

                # If there is one at the beginning of the line, tokenise it.
                else:
                    tokens.append(line[:token_match.end()])
                    line = line[token_match.end():]
            else:
                # Else there is unmatchable material here.
                # It ends where a skippable or token match starts, or at the end of the line.
                unmatchable_end = len(line)
                if skippable_match:
                    unmatchable_end = skippable_match.start()
                if token_match:
                    unmatchable_end = min(unmatchable_end, token_match.start())
                # Add it to unmatchable and discard from line.
                unmatchable.append(line[:unmatchable_end])
                line = line[unmatchable_end:]

    final_tokens = []

    while len(tokens) > 0:
        temp1 = tokens.pop(0)
        try:
            temp2 = tokens.pop(0)
            if temp2[0] == "'":
                temp1 += temp2
                final_tokens.insert(0, temp1)
                #print('a', temp1)
            else:
                final_tokens.insert(0, temp1)
                tokens.insert(0, temp2)
                #print('b', temp1, temp2)
        except:
            final_tokens.insert(0, temp1)
            #print('d', temp1)
        
    final_tokens = final_tokens[::-1]

    #print(final_tokens)
    #print(unmatchable)
    return final_tokens

In [5]:
f = open(IRONY_RAW_PATH + TRAIN, "r", encoding="utf-8")
final_irony = []
for line in f:
    final_irony.append(tokenise(line))
f.close()

with open(IRONY_INTERIM_PATH+TRAIN_INTERIM, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(final_irony)

In [6]:
final_irony[0]

['seeing',
 'ppl',
 'walking',
 'w',
 'crutches',
 'makes',
 'me',
 'really',
 'excited',
 'for',
 'the',
 'next',
 '3',
 'weeks',
 'of',
 'my',
 'life']

In [7]:
temp1 = pd.read_csv(IRONY_INTERIM_PATH+TRAIN_INTERIM, delimiter="\n", names=['tweet'])
temp2 = pd.read_csv(IRONY_RAW_PATH+TRAIN_LABELS)
irony_df = temp1.merge(temp2, left_index=True, right_index=True)

In [9]:
irony_df.shape #idk why there isn't 9000

(2861, 2)

In [10]:
f = open(EMOJI_RAW_PATH + TRAIN, "r", encoding="utf-8")
final_emoji = []
for line in f:
    final_emoji.append(tokenise(line))
f.close()

with open(EMOJI_INTERIM_PATH+TRAIN_INTERIM, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(final_emoji)

In [11]:
final_emoji[0]

['Sunday',
 'afternoon',
 'walking',
 'through',
 'Venice',
 'in',
 'the',
 'sun',
 'with',
 '@user',
 'Abbot',
 'Kinney',
 'Venice']

In [12]:
temp1 = pd.read_csv(EMOJI_INTERIM_PATH+TRAIN_INTERIM, delimiter="\n", names=['tweet'])
temp2 = pd.read_csv(EMOJI_RAW_PATH+TRAIN_LABELS)
emoji_df = temp1.merge(temp2, left_index=True, right_index=True)

In [13]:
emoji_df.shape #should be 45000


count_emoji = Counter()

for line in final_emoji:
    count_emoji.update(line)

count_emoji.most_common()

[('@user', 12189),
 ('the', 10345),
 ('to', 7645),
 ('I', 6535),
 ('a', 6100),
 ('my', 5965),
 ('in', 5687),
 ('and', 5389),
 ('with', 5048),
 ('of', 5016),
 ('you', 4772),
 ('for', 4698),
 ('California', 3992),
 ('this', 3563),
 ('at', 3454),
 ('The', 3385),
 ('is', 3381),
 ('on', 2598),
 ('love', 2134),
 ('amp', 2131),
 ('it', 1967),
 ('me', 1966),
 ('Los', 1916),
 ('San', 1865),
 ('Angeles', 1750),
 ('so', 1639),
 ('My', 1601),
 ('from', 1535),
 ('day', 1513),
 ('was', 1463),
 ('Beach', 1451),
 ('be', 1372),
 ('by', 1328),
 ('your', 1309),
 ('all', 1308),
 ('Happy', 1286),
 ('Vegas', 1273),
 ('out', 1237),
 ('that', 1214),
 ('up', 1178),
 ('are', 1168),
 ('today', 1166),
 ('night', 1165),
 ('our', 1151),
 ('time', 1135),
 ("I'm", 1115),
 ('Christmas', 1103),
 ('Las', 1053),
 ('This', 987),
 ('we', 983),
 ('like', 960),
 ('have', 955),
 ('one', 935),
 ('Park', 922),
 ('CA', 921),
 ('A', 909),
 ('Hollywood', 876),
 ('Love', 854),
 ('We', 849),
 ('but', 832),
 ('just', 798),
 ('good', 

In [14]:
emoji_df.head()

,tweet,12
0,"Sunday,afternoon,walking,through,Venice,in,the...",19
1,"Time,for,some,BBQ,and,whiskey,libations,Chomp,...",0
2,"Love,love,love,all,these,people,#friends,#bff,...",0
3,"Toys,R,Us",2
4,"Man,these,are,the,funniest,kids,ever,That,face...",11


In [18]:
temp1 = pd.read_csv(IRONY_INTERIM_PATH+TRAIN_INTERIM, delimiter="\n", names=['tweet'])
temp2 = pd.read_csv(IRONY_RAW_PATH+TRAIN_LABELS, names=['label'])
irony_df = temp1.merge(temp2, left_index=True, right_index=True)

In [22]:
irony_df

,tweet,label
0,"seeing,ppl,walking,w,crutches,makes,me,really,...",1
1,"look,for,the,girl,with,the,broken,smile,ask,he...",0
2,"Now,I,remember,why,I,buy,books,online,@user,#s...",1
3,"@user,@user,So,is,he,banded,from,wearing,the,c...",1
4,"Just,found,out,there,are,Etch,A,Sketch,apps,#o...",1
...,...,...
2857,"I,don't,have,to,respect,your,beliefs,I,only,ha...",0
2858,"Women,getting,hit,on,by,married,managers,at,@u...",1
2859,"@user,no,but,i,followed,you,and,i,saw,you,post...",0
2860,"@user,I,dont,know,what,it,is,but,I'm,in,love,y...",0
